In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/')

In [4]:
!ls "Intent classification"

preprocessed_test_data.csv  preprocessed_train_data.csv  test.csv  test_labels.csv  train.csv


In [5]:
!cp "Intent classification"/* /content/

In [45]:
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

In [46]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [47]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [50]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [55]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Remove web links
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove special characters, punctuation marks, and newlines
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra white spaces
    text = re.sub(r'\s+', ' ', text)

    # Remove stopwords
    text = ' '.join(word for word in text.split() if word.lower() not in stop)

    return text.lower()

In [56]:
train_data['comment_text'] = train_data['comment_text'].apply(clean_text)
train_data['comment_text'].head()

0    explanation edits made username hardcore metal...
1    daww matches background colour im seemingly st...
2    hey man im really trying edit war guy constant...
3    cant make real suggestions improvement wondere...
4                  sir hero chance remember page thats
Name: comment_text, dtype: object

In [57]:
test_data['comment_text'] = train_data['comment_text'].apply(clean_text)
test_data['comment_text'].head()

0    explanation edits made username hardcore metal...
1    daww matches background colour im seemingly st...
2    hey man im really trying edit war guy constant...
3    cant make real suggestions improvement wondere...
4                  sir hero chance remember page thats
Name: comment_text, dtype: object

In [62]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

In [63]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

def stem_lem(text):
    words = nltk.word_tokenize(text.lower())
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    stemmed = [stemmer.stem(word) for word in lemmatized]
    return ' '.join(stemmed)

In [64]:
train_data['comment_text'] = train_data['comment_text'].apply(stem_lem)
train_data['comment_text'].head()

0    explan edit made usernam hardcor metallica fan...
1    daww match background colour im seem stuck tha...
2    hey man im realli tri edit war guy constant re...
3    cant make real suggest improv wonder section s...
4                      sir hero chanc rememb page that
Name: comment_text, dtype: object

In [65]:
test_data['comment_text'] = train_data['comment_text'].apply(stem_lem)
test_data['comment_text'].head()

0    explan edit made usernam hardcor metallica fan...
1    daww match background colour im seem stuck tha...
2    hey man im realli tri edit war guy constant re...
3    cant make real suggest improv wonder section s...
4                      sir hero chanc rememb page that
Name: comment_text, dtype: object

In [67]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww match background colour im seem stuck tha...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im realli tri edit war guy constant re...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggest improv wonder section s...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chanc rememb page that,0,0,0,0,0,0


In [68]:
#train_data.to_csv('/content/preprocessed_train_data.csv', index=False)
#test_data.to_csv('/content/preprocessed_test_data.csv', index=False)

In [71]:
# Move files to destination folder
#!cp /content/preprocessed_train_data.csv '/content/drive/My Drive/Intent classification/'
#!cp /content/preprocessed_test_data.csv '/content/drive/My Drive/Intent classification/'

In [39]:
#!pip install sentence_transformers

In [40]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
sentences = [' '.join(sentence) for sentence in X]

In [44]:
sentences[0]

'[ \' e x p l a n a t i o n \' ,   \' w h y \' ,   \' t h e \' ,   \' e d i t s \' ,   \' m a d e \' ,   \' u n d e r \' ,   \' m y \' ,   \' u s e r n a m e \' ,   \' h a r d c o r e \' ,   \' m e t a l l i c a \' ,   \' f a n \' ,   \' w e r e \' ,   \' r e v e r t e d \' ,   \' ? \' ,   \' t h e y \' ,   \' w e r e \' ,   " n \' t " ,   \' v a n d a l i s m \' ,   \' , \' ,   \' j u s t \' ,   \' c l o s u r e \' ,   \' o n \' ,   \' s o m e \' ,   \' g a s \' ,   \' a f t e r \' ,   \' i \' ,   \' v o t e d \' ,   \' a t \' ,   \' n e w \' ,   \' y o r k \' ,   \' d o l l \' ,   \' f a c \' ,   \' . \' ,   \' a n d \' ,   \' p l e a s e \' ,   \' d o \' ,   " n \' t " ,   \' r e m o v e \' ,   \' t h e \' ,   \' t e m p l a t e \' ,   \' f r o m \' ,   \' t h e \' ,   \' t a l k \' ,   \' p a g e \' ,   \' s i n c e \' ,   \' i \' ,   " \' m " ,   \' r e t i r e d \' ,   \' n o w . 8 9 . 2 0 5 . 3 8 . 2 7 \' ]'

In [ ]:
sentence_embeddings = model.encode(sentences)

In [33]:
import ast
X = [ast.literal_eval(s) for s in X]
X = np.asarray(X)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (159571,) + inhomogeneous part.

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.1,random_state= 42)

In [22]:
X_train.shape,y_train.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the documents into a bag-of-words representation
X_train_tf = vectorizer.fit_transform(X_train)
X_test_processed = vectorizer.transform(X_test)
X_train_tf.shape

(106912, 167866)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfid = TfidfTransformer()
X_train_tfid = tfid.fit_transform(X_train_tf)
X_train_tfid.shape

(106912, 167866)

In [ ]:
y_train.shape

(106912, 6)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.multioutput import MultiOutputClassifier  # For multi-label classification
from sklearn.multioutput import MultiOutputRegressor  # For multi-label regression
from sklearn.naive_bayes import MultinomialNB

# Create an instance of the MultinomialNB class
mnb = MultinomialNB()

clf = MultiOutputClassifier(mnb)  # Replace YourClassifier with the desired classifier
clf.fit(X_train_tfid, y_train)


MultiOutputClassifier(estimator=MultinomialNB())

In [ ]:
# Make predictions
nb_prediction = clf.predict(X_test_processed)
nb_pred = clf.predict(X_train_tfid)

In [ ]:
print(" Test Accuracy {0:.2f}%".format(100*accuracy_score(nb_prediction,y_test)))
print(" Train Accuracy {0:.2f}%".format(100*accuracy_score(nb_pred, y_train)))
print(classification_report(nb_prediction,y_test))

 Test Accuracy 89.94%
 Train Accuracy 90.08%
              precision    recall  f1-score   support

           0       0.20      0.97      0.34      1077
           1       0.02      0.36      0.04        28
           2       0.17      0.94      0.28       503
           3       0.00      0.00      0.00         8
           4       0.10      0.82      0.18       324
           5       0.01      0.28      0.02        18

   micro avg       0.15      0.92      0.26      1958
   macro avg       0.08      0.56      0.14      1958
weighted avg       0.17      0.92      0.29      1958
 samples avg       0.01      0.02      0.01      1958



In [ ]:
logreg_clf = LogisticRegression(random_state=42,class_weight = 'balanced')

clf = MultiOutputClassifier(logreg_clf)  # Replace YourClassifier with the desired classifier
clf.fit(X_train_tfid, y_train)

MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   random_state=42))

In [ ]:
# Predicting Train and Test data
log_prediction = clf.predict(X_test_processed)
log_pred = clf.predict(X_train_tfid)

In [ ]:
print(" Test Accuracy {0:.2f}%".format(100*accuracy_score(log_prediction,y_test)))
print(" Train Accuracy {0:.2f}%".format(100*accuracy_score(log_pred, y_train)))
print(classification_report(log_prediction,y_test))

 Test Accuracy 75.59%
 Train Accuracy 91.24%
              precision    recall  f1-score   support

           0       0.90      0.46      0.61      9994
           1       0.92      0.18      0.29      2823
           2       0.95      0.36      0.52      7610
           3       0.94      0.03      0.06      4270
           4       0.93      0.32      0.48      7629
           5       0.86      0.10      0.17      4261

   micro avg       0.92      0.30      0.45     36587
   macro avg       0.92      0.24      0.36     36587
weighted avg       0.92      0.30      0.43     36587
 samples avg       0.09      0.05      0.06     36587



In [ ]:
rf_clf = RandomForestClassifier(n_estimators=10)

clf = MultiOutputClassifier(rf_clf)  # Replace YourClassifier with the desired classifier
clf.fit(X_train_tfid, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10))

In [ ]:
# Predicting Train and Test data
rf_prediction = clf.predict(X_test_processed)
rf_pred = clf.predict(X_train_tfid)

In [ ]:
print(" Test Accuracy {0:.2f}%".format(100*accuracy_score(rf_prediction,y_test)))
print(" Train Accuracy {0:.2f}%".format(100*accuracy_score(rf_pred, y_train)))
print(classification_report(rf_prediction,y_test))

 Test Accuracy 89.13%
 Train Accuracy 98.36%
              precision    recall  f1-score   support

           0       0.49      0.73      0.58      3433
           1       0.14      0.36      0.20       207
           2       0.48      0.82      0.60      1674
           3       0.08      0.55      0.14        22
           4       0.34      0.76      0.46      1179
           5       0.05      0.39      0.09        59

   micro avg       0.41      0.74      0.53      6574
   macro avg       0.26      0.60      0.35      6574
weighted avg       0.44      0.74      0.55      6574
 samples avg       0.04      0.05      0.04      6574



In [ ]:
xg_clf = XGBClassifier()

clf = MultiOutputClassifier(xg_clf)  # Replace YourClassifier with the desired classifier
clf.fit(X_train_tfid, y_train)


MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))

In [ ]:
# Predicting Train and Test data
xg_prediction = clf.predict(X_test_processed)
xg_pred = clf.predict(X_train_tfid)

In [ ]:
print(" Test Accuracy {0:.2f}%".format(100*accuracy_score(xg_prediction,y_test)))
print(" Train Accuracy {0:.2f}%".format(100*accuracy_score(xg_pred, y_train)))
print(classification_report(xg_prediction,y_test))

 Test Accuracy 90.57%
 Train Accuracy 94.31%
              precision    recall  f1-score   support

           0       0.63      0.78      0.70      4157
           1       0.40      0.38      0.39       564
           2       0.74      0.82      0.78      2565
           3       0.32      0.31      0.31       156
           4       0.57      0.74      0.65      2055
           5       0.23      0.53      0.32       208

   micro avg       0.61      0.75      0.67      9705
   macro avg       0.48      0.59      0.52      9705
weighted avg       0.62      0.75      0.68      9705
 samples avg       0.05      0.06      0.05      9705



##Deep Learning Model

In [ ]:
MAX_FEATURES = 5000 # number of words in the vocab

In [ ]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

In [ ]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length= 500,
                               output_mode='int')

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorized_text = vectorizer(X.values)

In [ ]:
vectorized_text

<tf.Tensor: shape=(159571, 500), dtype=int64, numpy=
array([[ 624,   61,  108, ...,    0,    0,    0],
       [   1,   39, 2432, ...,    0,    0,    0],
       [ 406,  420,   55, ...,    0,    0,    0],
       ...,
       [   1,    1,  360, ...,    0,    0,    0],
       [ 516,   37,   12, ...,    0,    0,    0],
       [   2,  115,   40, ...,    0,    0,    0]])>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
X

0         explanation why edits made under my username h...
1         daww he matches this background colour im seem...
2         hey man im really not trying edit war its just...
3         more i cant make any real suggestions improvem...
4         sir are my hero any chance remember what page ...
                                ...                        
159566    second time asking when view completely contra...
159567    should be ashamed yourself that horrible thing...
159568    spitzer umm theres no actual article prostitut...
159569    looks like was actually who put speedy have fi...
159570    i really dont think understand i came here my ...
Name: comment_text, Length: 159571, dtype: object

In [ ]:
y

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [ ]:
#!pip install transformers

In [ ]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(1600)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [ ]:
train = dataset.take(int(len(dataset)*.6))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.2))

##Sequential Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()
# Create the embedding layer
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(1, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='relu'))
# Final layer
model.add(Dense(6, activation='softmax'))

In [ ]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          160032    
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
 dense_1 (Dense)             (None, 2)                 4         
                                                                 
 dense_2 (Dense)             (None, 1)                 3         
                                                                 
 dense_3 (Dense)             (None, 6)                 12        
                                                        

In [ ]:
# Fit the model
#history = model.fit(vectorized_text, y, validation_split=0.33, epochs=1, batch_size=10)
history = model.fit(train, epochs=10, validation_data=val,batch_size=100)

Epoch 1/10
5984/5984 [==============================] - 1903s 318ms/step - loss: 0.1423 - accuracy: 0.9942 - val_loss: 0.1412 - val_accuracy: 0.9941
Epoch 2/10
5984/5984 [==============================] - 1934s 323ms/step - loss: 0.1415 - accuracy: 0.9942 - val_loss: 0.1404 - val_accuracy: 0.9942
Epoch 3/10
5984/5984 [==============================] - 1942s 325ms/step - loss: 0.1417 - accuracy: 0.9942 - val_loss: 0.1416 - val_accuracy: 0.9939
Epoch 4/10
5984/5984 [==============================] - 1937s 324ms/step - loss: 0.1416 - accuracy: 0.9943 - val_loss: 0.1409 - val_accuracy: 0.9940
Epoch 5/10
5984/5984 [==============================] - 1933s 323ms/step - loss: 0.1416 - accuracy: 0.9943 - val_loss: 0.1405 - val_accuracy: 0.9940
Epoch 6/10
5984/5984 [==============================] - 1943s 325ms/step - loss: 0.1415 - accuracy: 0.9942 - val_loss: 0.1405 - val_accuracy: 0.9940
Epoch 7/10
5984/5984 [==============================] - 1945s 325ms/step - loss: 0.1416 - accuracy: 0.9942

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 61ms/step


In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.0, Recall:0.0, Accuracy:0.2545090317726135


In [ ]:
#history_Test = model.fit(test, epochs=1, validation_data=val)

998/998 [==============================] - 445s 446ms/step - loss: 0.1480 - accuracy: 0.9938 - val_loss: 0.1430 - val_accuracy: 0.9940


In [ ]:
model.save('intent_model.h5')

In [ ]:
from google.colab import files

# Specify the file path you want to download
file_path = '/content/intent_model.h5'

# Download the file
files.download(file_path)

In [ ]:
'''
import pickle
# Convert the Keras model to a dictionary representation
model_dict = model.to_json()

# Save the model dictionary as a pickle file
with open('intent_model.pkl', 'wb') as f:
    pickle.dump(model_dict, f)
'''

"\nimport pickle\n# Convert the Keras model to a dictionary representation\nmodel_dict = model.to_json()\n\n# Save the model dictionary as a pickle file\nwith open('intent_model.pkl', 'wb') as f:\n    pickle.dump(model_dict, f)\n"

In [ ]:
#model.save_weights('intent_model_weights.h5')

In [ ]:
from keras.models import load_model
model2 = load_model('intent_model.h5')

0 toxic
1 severe_toxic
2 obscene
3 threat
4 insult
5 identity_hate

In [ ]:
temp=vectorizer('Yo ssup')
temp = np.expand_dims(temp, axis=0)

In [ ]:
model.predict(temp)

1/1 [==============================] - 0s 79ms/step


array([[0.37285644, 0.07939051, 0.20781665, 0.06611224, 0.19627038,
        0.07755368]], dtype=float32)

In [ ]:
temp=vectorizer('Daww He matches this background colour ')
temp = np.expand_dims(temp, axis=0)

In [ ]:
model.predict(temp)

1/1 [==============================] - 0s 50ms/step


array([[0.37285644, 0.07939051, 0.20781665, 0.06611224, 0.19627038,
        0.07755368]], dtype=float32)

In [ ]:
y_pred = (y_pred > 0.5)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support



           0       0.00      0.00      0.00      1276

           1       0.00      0.00      0.00       160

           2       0.00      0.00      0.00       728

           3       0.00      0.00      0.00        42

           4       0.00      0.00      0.00       663

           5       0.00      0.00      0.00        96



   micro avg       0.00      0.00      0.00      2965

   macro avg       0.00      0.00      0.00      2965

weighted avg       0.00      0.00      0.00      2965

 samples avg       0.00      0.00      0.00      2965


